# Комп'ютерний практикум №4
## Сидорський Володимир Сергійович (КА-21ф)
## Варіант 5

# Завдання

![Task_1](images/task_num_1.png)

![Task_2](images/task_num_2.png)

![Task_3](images/task_num_3.png)

# Програма

In [1]:
import numpy as np
import scipy


def method_CI(A):
    def calc_CI(A_trimmed):
        N = A_trimmed.shape[0]
        lam_max = np.linalg.eigvals(A_trimmed).max()
        return (lam_max - N) / (N-1)

    CIs = [calc_CI(np.delete(np.delete(A, i, axis=0), i, axis=1)) for i in range(A.shape[0])]
    sorted_idxs_by_CI = np.argsort(CIs)
    outlier_i, outlier_j = np.sort(sorted_idxs_by_CI[:2])
    
    print(f"Викид за методом CI: D{(outlier_i+1, outlier_j+1)} і його оберений. значення = {A[outlier_i, outlier_j]}")



def method_corr(A):
    N = A.shape[0]
    
    corr = []
    for i in range(A.shape[0]):
        row_x = A[i]
        std_x = np.std(row_x)
        m_corr = 0
        for i_y in range(A.shape[0]):
            if i_y == i: continue
            row_y = A[i_y]
            std_y = np.std(row_y)
            m_corr += (1.0 / (N - 1)) * np.sum((row_x - row_x.mean()) * (row_y - row_y.mean())) / (std_x * std_y) 
        m_corr /= N - 1
        corr.append(m_corr)
    outlier_i = np.argmin(corr)
    
    corr = []
    for j in range(A.shape[1]):
        col_x = A[:, j]
        std_x = np.std(col_x)
        m_corr = 0
        for j_y in range(A.shape[1]):
            if j_y == j: continue
            col_y = A[:, j_y]
            std_y = np.std(col_y)
            m_corr += 1.0 / (N - 1) * np.sum((col_x - col_x.mean()) * (col_y - col_y.mean())) / std_x / std_y 
        m_corr /= N - 1
        corr.append(m_corr)
    outlier_j = np.argmin(corr)
                
    print(f"Викид за методом кореляцій між рядками і стовпчиками D{(outlier_i+1, outlier_j+1)} і його оберений. значення = {A[outlier_i, outlier_j]}")


def method_chi_sq(A):
    N = A.shape[0]
    Delta = np.empty((N, N))
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            t_ij = A[i,:].sum() * A[:,j].sum() / A.sum()
            Delta[i,j] = (A[i,j] - t_ij)**2 / t_ij

    delta_std = Delta.flatten().std()
    delta_mean = Delta.mean()
    print("Викиди за методом Хі-квадрат:")
    for i in range(N):
        for j in range(N):
            if Delta[i, j] > delta_mean + delta_std * 1.645 / np.sqrt(N):
                print(f"D{i+1, j+1} і його обернений. значення = {A[i,j]}")

def method_transitive(A):
    N = A.shape[0]
    S = np.empty((N, N))
    outlier_i = -1
    outlier_j = -1
    S_ij_max = float("-inf")
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            S_ij = np.sum(A[i,j] * A[j, :] / A[i, :] + A[i,:] / A[i, j] / A[j, :] - 2)
            if S_ij > S_ij_max:
                S_ij_max = S_ij
                outlier_i = i
                outlier_j = j
    print(f"Викид за методом Транзитивностей D{(outlier_i+1, outlier_j+1)} і його обернений. value = {A[outlier_i, outlier_j]}")


def method_M_OutFlow(A):
    N = A.shape[0]
    Phi_pos = np.empty(N)
    Phi_neg = np.empty(N)
    for i in range(A.shape[0]):
        Phi_pos[i] = (A[i, :] > 1).sum()
        Phi_neg[i] = (A[:, i] > 1).sum()

    gamma_max = float("-inf")
    max_val = float("-inf")
    outlier_i = -1
    outlier_j = -1

    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i, j] <= 1: continue
                
            val = max(Phi_pos[j] - Phi_pos[i], Phi_neg[i] - Phi_neg[j])
            gamma_ = np.log(A[i, j]) - np.log(A[i, :] * A[:, j])
            mask = np.ones(gamma_.shape[0])
            mask[i] = 0.0
            mask[j] = 0.0
            gamma = 1.0 / (N - 2.0) * np.sum(gamma_ * mask)
            
            
            if val > max_val or (val == max_val and gamma >= gamma_max):
                outlier_i = i
                outlier_j = j
                max_val = val
                gamma_max = gamma

    print(f"Викид за методом M_OutFlow D{(outlier_i+1, outlier_j+1)} і його обернений. значення = {A[outlier_i, outlier_j]}")

# Виклик програми

Вхідна матриця `A`

In [2]:
A = np.array([
    [1, 0.5, 0.25, 2, 1, 0.5, 2],
    [2, 1, 0.25, 0.5, 1, 0.5, 2],
    [4, 4, 1, 2, 5, 2, 0.142],
    [0.5, 2, 0.5, 1, 2, 1, 4],
    [1, 1, 0.2, 0.5, 1, 0.5, 2],
    [2, 2, 0.5, 1, 2, 1, 4],
    [0.5, 0.5, 7, 0.25, 0.5, 0.25, 1]
])
A

array([[1.   , 0.5  , 0.25 , 2.   , 1.   , 0.5  , 2.   ],
       [2.   , 1.   , 0.25 , 0.5  , 1.   , 0.5  , 2.   ],
       [4.   , 4.   , 1.   , 2.   , 5.   , 2.   , 0.142],
       [0.5  , 2.   , 0.5  , 1.   , 2.   , 1.   , 4.   ],
       [1.   , 1.   , 0.2  , 0.5  , 1.   , 0.5  , 2.   ],
       [2.   , 2.   , 0.5  , 1.   , 2.   , 1.   , 4.   ],
       [0.5  , 0.5  , 7.   , 0.25 , 0.5  , 0.25 , 1.   ]])

Застосуємо усі необіхдні методи для знаходження викидів

In [3]:
method_CI(A)

Викид за методом CI: D(3, 7) і його оберений. значення = 0.142


In [4]:
method_corr(A)

Викид за методом кореляцій між рядками і стовпчиками D(7, 3) і його оберений. значення = 7.0


In [5]:
method_chi_sq(A)

Викиди за методом Хі-квадрат:
D(3, 7) і його обернений. значення = 0.142
D(7, 3) і його обернений. значення = 7.0


In [6]:
method_transitive(A)

Викид за методом Транзитивностей D(3, 7) і його обернений. value = 0.142


In [7]:
method_M_OutFlow(A)

Викид за методом M_OutFlow D(7, 3) і його обернений. значення = 7.0


# Висновки

В даній роботі, було імплементовано та досліджено методи знаходження найбільш неузгоджених експертних оцінок парних порівнянь: `СІ для укороченої МПП`, `кореляції між рядками і стовпчиками МПП`, `метод, що використовує критерій Хі-квадрат`, `метод транзитивностей` та `метод потоків M_OutFlow`. Всі методи знайшли один і той самий викид або його обернений або обидва. Всі використані методи було запрограмовано, текст програми наведено в звіті.